In [ ]:
import os
import random
import time

import dask
from dask.distributed import Client

client = Client("tcp://dask-alheld-d8a070f6-a.af-jupyter:8786")  # update to your client

caches = ['192.170.240.141', '192.170.240.142', '192.170.240.143',
          '192.170.240.144', '192.170.240.145', '192.170.240.146',
          '192.170.240.147', '192.170.240.148']


with open("ttbar_for_xcache_tests.txt" ) as f:
    filenames = [l.strip() for l in f.readlines()]


def run_xrdcp(idx):
    # pick one of the three blocks below to configure
    
    # use same xcache for all files (deterministically, varies by worker, equal to k=1 below)
    # xcache_to_use = caches[idx%8]
    # paths = ["root://" + xcache_to_use + c[22:] for c in filenames]

    # fully randomize list of xcaches for each file, varies across workers randomly (equal to k=8 below)
    # paths = []
    # random_xcaches = [caches[random.randint(1, 8)%8] for _ in range(len(filenames))]
    # for i, c in enumerate(filenames):
    #     paths.append("root://" + random_xcaches[i] + c[22:])

    # give each worker k xcaches, randomize files pulled out from just these
    paths = []
    xcaches_for_worker = random.sample(caches, k=4)  # set k here
    assigned_xcaches = random.choices(xcaches_for_worker, k=len(filenames))
    for i, c in enumerate(filenames):
        paths.append("root://" + assigned_xcaches[i] + c[22:])

    t0 = time.perf_counter()
    for fname in paths:
        # print(f"xrdcp {fname} /dev/null -f")
        os.system(f"xrdcp {fname} /dev/null -f")
    t1 = time.perf_counter()
    time_finished = datetime.datetime.now()
    return {"runtime": t1-t0, "time_finished": time_finished}


NUM_TASKS = 1000  # total number of tasks (to run in parallel, given enough workers are available)
tasks = [dask.delayed(run_xrdcp)(i) for i in range(NUM_TASKS)]
_ = dask.compute(*tasks)